In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K
import numpy as np
import os
from utils import preprocess_images

In [ ]:
from models import Encoder as buildVision
from models import M as buildMemory

In [ ]:
import mdn

In [ ]:
V = buildVision()

In [ ]:
M = buildMemory()

In [ ]:
M.layers

In [ ]:
get_hidden = K.function(M.layers[0].input, M.layers[0].output)

In [ ]:
some_z = np.load('./sausage/z_states/z_state0_1005.npy')

In [ ]:
some_a = np.array([1,0,0])

In [ ]:
combined = np.concatenate([some_z, some_a], axis=0)

In [ ]:
#inputs = np.zeros((128, *combined.shape))
#inputs[0] = combined

In [ ]:
#inputs.shape

In [ ]:
M.reset_states()

In [ ]:
M.layers[0]

In [ ]:
foo = get_hidden(tf.expand_dims(tf.expand_dims(combined, 0), 0))

In [ ]:
bar = M(tf.expand_dims(c, 0))

In [ ]:
np.apply_along_axis(mdn.sample_from_output, 1, bar[0], 32, 5, temp=1.0).shape

In [ ]:
class Controller():
    def __init__(self, input_size, output_size):
        self._in = input_size
        self._out = output_size
        self.W = np.random.randn(input_size, output_size)
    
    def clip(self, x, lo=0.0, hi=1.0):
        return np.minimum(np.maximum(x, lo), hi)
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def __call__(self, obs):
        action = np.dot(obs, self.W)
        
        action[0] = np.tanh(action[0])
        action[1] = self.sigmoid(action[1])
        action[2] = self.clip(np.tanh(action[2]))
        
        return action
    
    def set_weights(self, W):
        # assume W is flat.
        self.W = np.reshape(W, self.W.shape)
        
    def randomly_init(self):
        self.W = np.random.randn(*self.W.shape)
       
    @property
    def shape(self):
        return self.W.shape

In [ ]:
controller = Controller(32+256, 3)

In [ ]:
foo[0][0].shape

In [ ]:
controller(np.concatenate([foo[0][0], some_z], axis=0))

In [ ]:
import gym

In [ ]:
env = gym.make("CarRacing-v0")

In [ ]:
state = preprocess_images(env.reset())
env.close()

In [ ]:
def rollout(controller):
    ims = []
    state = preprocess_images(env.reset())
    
    M.reset_states()
    h = np.zeros(256)
    done = False
    cumulative_reward = 0
    
    while not done:
        _state = np.zeros((128, 64, 64, 3))
        _state[0] = state
        ims.append(state)
        z = V(_state)[0][0] #extract first from batch and sequence
        
        # combine V latent space with M hidden space 
        combined = np.concatenate([z, h], axis=0)
        
        a = controller(combined)
        
        state, reward, done, info = env.step(a)
        state = preprocess_images(state)
        
        cumulative_reward += reward
        
        # get factored gaussians
        # by feeding current latent_state + action
        z = M(tf.expand_dims(tf.expand_dims(np.concatenate([z, a]), 0), 0))
        
        # sample from factored gaussians
        # 32 = output_dims
        # 5  = num_mixtures
        z = np.apply_along_axis(mdn.sample_from_output, 1, z[0], 32, 5, temp=1.0).squeeze()

        # extract hidden state from LSTM
        h = get_hidden(tf.expand_dims(tf.expand_dims(np.concatenate([z, a], 0), 0), 0)).squeeze()
    
    env.close()
    return cumulative_reward, ims

In [ ]:
r, ims = rollout(controller)

In [ ]:
r

In [ ]:
import matplotlib.pyplot as plt
from IPython import display

In [ ]:
def show_state(env, step=0, name="", info="", image=None):
    """Fn to visualize the agent playing the game in a notebook
    """
    plt.figure(10)
    plt.clf()
    if image is not None:
        im = image
    else:
        im = env.render(mode="rgb_array")[0]
    plt.imshow(im)
    plt.title("{} | Step: {} {}".format(name, step, info))
    plt.axis('off')
    display.clear_output(wait=True)
    display.display(plt.gcf())

In [ ]:
for i in ims:
    show_state(None, image=i[0])

In [ ]:
np.save("AICAR.npy", ims)